In [1]:
import torch
import networkx as nx
import matplotlib.pyplot as plt
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def initialize_model():
    """Initialize the tokenizer and GPT-2 model."""
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2", padding_side="left")
    model = GPT2LMHeadModel.from_pretrained("gpt2").half()
    model.to(device)
    tokenizer.pad_token = tokenizer.eos_token
    return tokenizer, model

def create_knowledge_graph():
    """Create an expanded knowledge graph with philosophical concepts."""
    G = nx.Graph()
    
    # Core concepts from original
    core_concepts = ["inner_beauty", "mindfulness", "well_being"]
    for concept in core_concepts:
        G.add_node(concept, type="core")
    
    # New concepts from the documents, expanded
    philosophical_concepts = {
        "beauty_maximization": "Focusing on and channeling inner beauty",
        "full_expression": "Understanding and channeling authentic self",
        "meditation": "Various techniques for mind training",
        "pragmatism": "Practical approach to philosophy and decision-making",
        "complexity": "Understanding and navigating complex decisions",
        "self_monitoring": "Awareness and adjustment of behavior",
        "will_to_power": "Expression of strength within bounds",
        "tolerance": "Accepting different conditions and viewpoints",
        "intention": "Purpose behind actions",
        "silence": "Different types of meaningful quiet",
        "religious_perspective": "Non-dogmatic approach to spirituality",
        "intelligent_speech": "Speaking thoughtfully to convey depth and precision",
        "colloquial_speech": "Speaking informally to connect in everyday scenarios",
        "lack_of_logic_in_desire": "How passions diverge from rational thought",
        "ink_blot_effect": "How simple visuals evoke complex emotions and stories",
        "unique_people_descriptions": "Capturing the distinctiveness of individuals",
        "satori": "Understanding sudden enlightenment or awakening",
        "never_impressed": "The overshadowing power of death compared to life events",
        "satisfied_love": "Romantic satisfaction through timeless ideals",
        "inaccessibility_of_real": "Struggles with distinguishing real from simulations",
        "abstraction_importance": "The niche but critical role of abstraction in life"
    }

    # Add nodes with attributes
    for concept, description in philosophical_concepts.items():
        G.add_node(concept, description=description, type="philosophical")

    # Expanded relationships
    relationships = [
        ("beauty_maximization", "inner_beauty", "enhances"),
        ("meditation", "mindfulness", "develops"),
        ("mindfulness", "well_being", "improves"),
        ("full_expression", "inner_beauty", "manifests"),
        ("self_monitoring", "well_being", "maintains"),
        ("pragmatism", "complexity", "manages"),
        ("will_to_power", "full_expression", "drives"),
        ("tolerance", "well_being", "supports"),
        ("intention", "mindfulness", "guides"),
        ("silence", "meditation", "facilitates"),
        ("religious_perspective", "tolerance", "promotes"),
        ("meditation", "self_monitoring", "enhances"),
        ("complexity", "intention", "influences"),
        ("pragmatism", "will_to_power", "moderates"),
        ("intelligent_speech", "inner_beauty", "expresses"),
        ("colloquial_speech", "mindfulness", "engages"),
        ("lack_of_logic_in_desire", "self_monitoring", "challenges"),
        ("ink_blot_effect", "complexity", "illustrates"),
        ("unique_people_descriptions", "mindfulness", "captures"),
        ("satori", "meditation", "achieves"),
        ("never_impressed", "will_to_power", "questions"),
        ("satisfied_love", "well_being", "enhances"),
        ("inaccessibility_of_real", "complexity", "underscores"),
        ("abstraction_importance", "pragmatism", "contrasts")
    ]
    
    # Add edges with relationships
    for source, target, relationship in relationships:
        G.add_edge(source, target, relationship=relationship)
    
    return G

def visualize_graph(G, filename="knowledge_graph.png"):
    """Visualize the knowledge graph and save it to a file."""
    plt.figure(figsize=(15, 10))
    
    # Create layout
    pos = nx.spring_layout(G, k=1, iterations=50)
    
    # Draw nodes
    nx.draw_networkx_nodes(G, pos, 
                          node_color=['lightblue' if G.nodes[node]['type'] == 'core' else 'lightgreen' 
                                      for node in G.nodes()],
                          node_size=2000)
    
    # Draw edges
    nx.draw_networkx_edges(G, pos)
    
    # Draw labels
    nx.draw_networkx_labels(G, pos)
    
    # Draw edge labels
    edge_labels = nx.get_edge_attributes(G, 'relationship')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
    
    plt.title("Knowledge Graph Visualization")
    plt.axis('off')
    plt.tight_layout()
    
    # Save the plot
    plt.savefig(filename)
    plt.close()

def retrieve_from_graph(graph, query):
    """Enhanced retrieval from knowledge graph based on input query."""
    relevant_nodes = set()
    relevant_edges = set()
    
    # Convert query to lowercase for better matching
    query_lower = query.lower()
    query_terms = query_lower.split()
    
    # Check for relevant nodes based on query terms
    for node in graph.nodes():
        node_str = str(node).lower().replace('_', ' ')
        if any(term in node_str or node_str in term for term in query_terms):
            relevant_nodes.add(node)
            # Add immediate neighbors for context
            relevant_nodes.update(graph.neighbors(node))
    
    # Retrieve edges between relevant nodes
    for node in relevant_nodes:
        for neighbor in graph.neighbors(node):
            if neighbor in relevant_nodes:
                edge_data = graph.get_edge_data(node, neighbor)
                relevant_edges.add((node, neighbor, edge_data['relationship']))
    
    return list(relevant_nodes), list(relevant_edges)

def generate_with_graph_knowledge(query, model, tokenizer, graph):
    """Generate a response based on a query and relevant knowledge from the graph."""
    relevant_nodes, relevant_edges = retrieve_from_graph(graph, query)
    
    # Create a more structured context
    context = f"Query: {query}\n\nRelevant Concepts:\n"
    
    # Add node information
    for node in relevant_nodes:
        description = graph.nodes[node].get('description', '')
        if description:
            context += f"• {node}: {description}\n"
        else:
            context += f"• {node}\n"
    
    # Add relationship information
    if relevant_edges:
        context += "\nRelationships:\n"
        for source, target, relationship in relevant_edges:
            context += f"• {source} {relationship} {target}\n"
    
    # Generate response with limited length to avoid repetition
    inputs = tokenizer(context, return_tensors="pt", padding=True, truncation=True).to(device)
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,  # Reduced to avoid repetition
        num_return_sequences=1,
        pad_token_id=tokenizer.pad_token_id,
        no_repeat_ngram_size=3,  # Prevent repetition of 3-grams
        temperature=0.7  # Add some randomness
    )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def save_graph(graph, filepath="knowledge_graph.pkl"):
    """Save the knowledge graph to a file."""
    import pickle
    with open(filepath, "wb") as f:
        pickle.dump(graph, f)

def load_graph(filepath="knowledge_graph.pkl"):
    """Load the knowledge graph from a file."""
    import pickle
    with open(filepath, "rb") as f:
        return pickle.load(f)

def main():
    """Main processing loop."""
    save_path = "knowledge_graph.pkl"
    
    try:
        graph = load_graph(save_path)
        print("Knowledge graph loaded successfully.")
    except FileNotFoundError:
        graph = create_knowledge_graph()
        save_graph(graph, save_path)
        print("Knowledge graph created and saved.")
    
    # Visualize the graph
    visualize_graph(graph)
    print("Graph visualization saved as 'knowledge_graph.png'")
    
    tokenizer, model = initialize_model()

    # Example queries
    queries = [
        "How can mindfulness improve well-being and inner beauty?",
        "What is the relationship between self-monitoring and meditation?",
        "How does pragmatism help manage complexity in decision-making?",
        "What is beauty?",
        "What holds back full expression?"
    ]

    responses = []
    for query in queries:
        response = generate_with_graph_knowledge(query, model, tokenizer, graph)
        responses.append(response)

    return responses

if __name__ == "__main__":
    try:
        outputs = main()
        for i, output in enumerate(outputs, 1):
            print(f"\nResponse {i}:")
            print(output)
    except Exception as e:
        print(f"An error occurred: {e}")


Using device: cpu
Knowledge graph loaded successfully.
Graph visualization saved as 'knowledge_graph.png'


/opt/homebrew/Caskroom/miniforge/base/envs/nunu24/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



Response 1:
Query: How can mindfulness improve well-being and inner beauty?

Relevant Concepts:
• mindfulness
• meditation: Various techniques for mind training
• intention: Purpose behind actions
• beauty_maximization: Focusing on and channeling inner beauty
• well_being
• inner_beauty
• full_expression: Understanding and channeling authentic self

Relationships:
• inner_beauty enhances beauty_maximization
• inner_beauty manifests full_expression
• mindfulness improves well_being
• intention guides mindfulness
• mindfulness develops meditation
• meditation develops mindfulness
• beauty_maximization enhances inner_beauty
• mindfulness guides intention
• full_expression manifests inner_beauty
• well_being improves mindfulness

• mindful practices

How can mindfulness enhance well-life and inner happiness?
.

.

Response 2:
Query: What is the relationship between self-monitoring and meditation?

Relevant Concepts:
• mindfulness
• meditation: Various techniques for mind training
• pragma